In [1]:
import numpy as np
import pandas as pd

In [3]:
df1 = pd.DataFrame({
    'key': list('bbacaab'),
    'data1': range(7),
})
df1

,data1,key
0,0,b
1,1,b
2,2,a
3,3,c
4,4,a
5,5,a
6,6,b


In [4]:
df2 = pd.DataFrame({
    'key': list('abd'),
    'data2': range(3),
})
df2

,data2,key
0,0,a
1,1,b
2,2,d


### 默认合并

* 使用数据集中均有的列名
* 内联接

In [19]:
pd.merge(df1, df2)

,data1,key,data2
0,0,b,1
1,1,b,1
2,6,b,1
3,2,a,0
4,4,a,0
5,5,a,0


### 指定连接用列（推荐）

In [6]:
pd.merge(df1, df2, on='key')

,data1,key,data2
0,0,b,1
1,1,b,1
2,6,b,1
3,2,a,0
4,4,a,0
5,5,a,0


如果列名不同，可以分开指定

In [14]:
df3 = df1.copy()
df3.columns = df3.columns.map(lambda x: 'l' + x)
df3

,ldata1,lkey
0,0,b
1,1,b
2,2,a
3,3,c
4,4,a
5,5,a
6,6,b


In [15]:
df4 = df2.copy()
df4.columns = df4.columns.map(lambda x: 'r' + x)
df4

,rdata2,rkey
0,0,a
1,1,b
2,2,d


In [17]:
pd.merge(df3, df4, left_on='lkey', right_on='rkey')

,ldata1,lkey,rdata2,rkey
0,0,b,1,b
1,1,b,1,b
2,6,b,1,b
3,2,a,0,a
4,4,a,0,a
5,5,a,0,a


如果某侧（或两侧）连接键在索引中，使用`left_index=True`或`right_index=True`指定

In [26]:
df5 = df2.copy()
df5.index=df2['key']
df5 = df5.drop(columns='key')
df5

,data2
key,
a,0
b,1
d,2


In [27]:
pd.merge(df3, df5, left_on='lkey', right_index=True)

,ldata1,lkey,data2
0,0,b,1
1,1,b,1
6,6,b,1
2,2,a,0
4,4,a,0
5,5,a,0


对于至少一侧使用index作为键值的合并，可以使用快捷方式`DataFrame.join`。
当`on`参数存在时，用本侧对应列于对侧索引合并。否则用两侧索引合并。

** 默认左连接 **

In [43]:
df3.join(df5, on='lkey')

,ldata1,lkey,data2
0,0,b,1.0
1,1,b,1.0
2,2,a,0.0
3,3,c,NaN
4,4,a,0.0
5,5,a,0.0
6,6,b,1.0


### 指定连接

* `inner`
* `outer`
* `left`
* `right`

In [21]:
pd.merge(df1, df2, on='key', how='inner')

,data1,key,data2
0,0,b,1
1,1,b,1
2,6,b,1
3,2,a,0
4,4,a,0
5,5,a,0


In [22]:
pd.merge(df1, df2, on='key', how='outer')

,data1,key,data2
0,0.0,b,1.0
1,1.0,b,1.0
2,6.0,b,1.0
3,2.0,a,0.0
4,4.0,a,0.0
5,5.0,a,0.0
6,3.0,c,NaN
7,NaN,d,2.0


In [23]:
pd.merge(df1, df2, on='key', how='left')

,data1,key,data2
0,0,b,1.0
1,1,b,1.0
2,2,a,0.0
3,3,c,NaN
4,4,a,0.0
5,5,a,0.0
6,6,b,1.0


In [24]:
pd.merge(df1, df2, on='key', how='right')

,data1,key,data2
0,0.0,b,1
1,1.0,b,1
2,6.0,b,1
3,2.0,a,0
4,4.0,a,0
5,5.0,a,0
6,NaN,d,2


### 多对多连接

并无特殊处理，直接调用即可。结果是行的笛卡尔积。

In [28]:
df6 = pd.DataFrame({
    'key': list('ababd'),
    'data2': range(5),
})
df6

,data2,key
0,0,a
1,1,b
2,2,a
3,3,b
4,4,d


In [29]:
df1

,data1,key
0,0,b
1,1,b
2,2,a
3,3,c
4,4,a
5,5,a
6,6,b


In [31]:
pd.merge(df1, df6, on='key')

,data1,key,data2
0,0,b,1
1,0,b,3
2,1,b,1
3,1,b,3
4,6,b,1
5,6,b,3
6,2,a,0
7,2,a,2
8,4,a,0
9,4,a,2


### 按多键连接

传入多个键即可。相当于多键值形成元组作为连接键

In [32]:
left = pd.DataFrame({
    'key1': ['foo', 'foo', 'bar'],
    'key2': ['one', 'two', 'one'],
    'lval': range(3)
})
right = pd.DataFrame({
    'key1': ['foo', 'foo', 'bar', 'bar'],
    'key2': ['one', 'one', 'one', 'tow'],
    'lval': range(4, 8)
})

In [33]:
left

,key1,key2,lval
0,foo,one,0
1,foo,two,1
2,bar,one,2


In [34]:
right

,key1,key2,lval
0,foo,one,4
1,foo,one,5
2,bar,one,6
3,bar,tow,7


In [35]:
pd.merge(left, right, on=['key1', 'key2'])

,key1,key2,lval_x,lval_y
0,foo,one,0,4
1,foo,one,0,5
2,bar,one,2,6


### 列后缀

上例中两个数值列重名（均为`lval`），被自动重命名为`lval_x`，`lval_y`。后缀可以定义：

In [36]:
pd.merge(left, right, on=['key1', 'key2'], suffixes=['_left', '_right'])

,key1,key2,lval_left,lval_right
0,foo,one,0,4
1,foo,one,0,5
2,bar,one,2,6
